In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/data_cleaned.csv')
df.head()

,Unnamed: 0,Id,categories,User_id,review_helpfulness,review_score,review_text,processed_text,tokens
0,0,1882931173,['Comics & Graphic Novels'],AVCGYZL8FQQTD,7/7,4.0,This is only for Julie Strain fans. It's a col...,this is only for julie strain fans its a colle...,"['julie', 'strain', 'fan', 'collection', 'phot..."
1,1,0826414346,['Biography & Autobiography'],A30TK6U7DNS82R,10/10,5.0,I don't care much for Dr. Seuss but after read...,i dont care much for dr seuss but after readin...,"['dont', 'care', 'much', 'dr', 'seuss', 'readi..."
2,2,0826414346,['Biography & Autobiography'],A3UH4UZ4RSVO82,10/11,5.0,"If people become the books they read and if ""t...",if people become the books they read and if th...,"['people', 'become', 'book', 'read', 'child', ..."
3,3,0826414346,['Biography & Autobiography'],A2MVUWT453QH61,7/7,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",theodore seuss geisel aka quotdr seussquot wa...,"['theodore', 'seuss', 'geisel', 'aka', 'quotdr..."
4,4,0826414346,['Biography & Autobiography'],A22X4XUPKF66MR,3/3,4.0,Philip Nel - Dr. Seuss: American IconThis is b...,philip nel dr seuss american iconthis is basi...,"['philip', 'nel', 'dr', 'seuss', 'american', '..."


In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model_name = "bhadresh-savani/bert-base-uncased-emotion"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

def get_emotion_bert(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        # Get the logits from the model
        outputs = model(**inputs)
        logits = outputs.logits
        # Get the predicted emotion
        emotion = torch.argmax(logits, dim=1).item()
    return emotion

df['review_tokens'] = df['tokens'].apply(lambda tokens: " ".join(tokens))
df['emotions'] = df['review_tokens'].apply(get_emotion_bert)

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

c:\Users\annma\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\annma\.cache\huggingface\hub\models--bhadresh-savani--bert-base-uncased-emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

c:\Users\annma\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pickle

# Encode emotion feature using One-Hot Encoding
encoder = OneHotEncoder(sparse=False)
emotion_encoded = encoder.fit_transform(df[['emotions']])
print("One-Hot Encoded emotions:\n", emotion_encoded)
emotion_feature_names = [f"emotion_{i}" for i in range(emotion_encoded.shape[1])]

# Add one-hot encoded emotion features to the reviews dataframe
emotion_df = pd.DataFrame(emotion_encoded, columns=emotion_feature_names)
reviews_df = pd.concat([df, emotion_df], axis=1)
# Save encoder to a pkl file
with open("../model/emotion_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("Encoder saved successfully.")